In [1]:
from ultrack.core.database import NodeDB, get_node_values
from ultrack.core.interactive import add_new_node
from ultrack.config import MainConfig
from pathlib import Path
import numpy as np

from sqlalchemy import create_engine, inspect, text

In [2]:
working_directory = Path('/home/teun.huijben/Documents/data/Akila/20241003/neuromast4_t851/adjusted/')
db_filename_new = 'data_v1.db'

config_adjusted = MainConfig()
config_adjusted.data_config.working_dir = working_directory
config_adjusted.data_config.database_file_name = db_filename_new

In [ ]:
index = 175000014
field = NodeDB.pickle

index = [int(index)]

val = get_node_values(config_adjusted.data_config,
        indices=index,
        values=field)

val

# Add columns 1by1

Get column names

In [ ]:
engine = create_engine(config_adjusted.data_config.database_path)
inspector = inspect(engine)

columns_info = inspector.get_columns('nodes')
column_names = [col['name'] for col in columns_info]

print(column_names)
print(len(column_names))

# print(' ')
# for i,c in enumerate(column_names):
#     offset = 79
#     if i >=3:
#         offset += 1
#     print(i+offset,c)

add missing columns 1 by 1

In [ ]:
expected_columns = {
    'frontier': 'FLOAT DEFAULT -1.0',
    'height': 'FLOAT DEFAULT -1.0',
    'features': 'FLOAT DEFAULT -1.0',
    'node_prob': 'FLOAT DEFAULT -1.0',
}

engine = create_engine(config_adjusted.data_config.database_path)
inspector = inspect(engine)
existing_columns = [col['name'] for col in inspector.get_columns('nodes')]

for col_name, col_definition in expected_columns.items():
    if col_name not in existing_columns:
        alter_stmt = f"ALTER TABLE nodes ADD COLUMN {col_name} {col_definition};"
        with engine.begin() as conn:  # This starts a transaction.
            conn.execute(text(alter_stmt))
        print(f"Added missing column: {col_name}")


check frontier value

In [ ]:
node_id = 175000014
fr = get_node_values(config_adjusted.data_config, [int(node_id)], NodeDB.frontier)
print(fr)


# Add all missing columns

Check all columns: 

In [3]:
def get_expected_columns(nodedb, engine):
    expected_columns = {}
    for column in nodedb.__table__.columns:
        # Compile the column type to its SQL representation.
        col_type_str = column.type.compile(dialect=engine.dialect)
        
        # Try to extract a default value if one is specified.
        default_value = None
        if column.default is not None and hasattr(column.default, "arg"):
            default_value = column.default.arg

        # Build the definition string.
        col_definition = col_type_str
        if default_value is not None:
            col_definition += f" DEFAULT {default_value}"
        
        expected_columns[column.name] = col_definition
    return expected_columns

engine = create_engine(config_adjusted.data_config.database_path)
inspector = inspect(engine)

expected_columns = get_expected_columns(NodeDB, engine)
existing_columns = [col['name'] for col in inspector.get_columns('nodes')]
print('expected columns',expected_columns)

for col_name, col_definition in expected_columns.items():
    if col_name not in existing_columns:
        alter_stmt = f"ALTER TABLE nodes ADD COLUMN {col_name} {col_definition};"
        with engine.begin() as conn:  # This starts a transaction.
            conn.execute(text(alter_stmt))
        print(f"Added missing column: {col_name}")

expected columns {'t': 'INTEGER', 'id': 'BIGINT', 'parent_id': 'BIGINT DEFAULT -1', 'hier_parent_id': 'BIGINT DEFAULT -1', 't_node_id': 'INTEGER', 't_hier_id': 'INTEGER', 'z': 'FLOAT', 'y': 'FLOAT', 'x': 'FLOAT', 'z_shift': 'FLOAT DEFAULT 0.0', 'y_shift': 'FLOAT DEFAULT 0.0', 'x_shift': 'FLOAT DEFAULT 0.0', 'area': 'INTEGER', 'frontier': 'FLOAT DEFAULT -1.0', 'height': 'FLOAT DEFAULT -1.0', 'selected': 'BOOLEAN DEFAULT False', 'pickle': 'BLOB', 'features': 'BLOB', 'node_prob': 'FLOAT DEFAULT -1.0', 'segm_annot': 'VARCHAR(14) DEFAULT 0', 'node_annot': 'VARCHAR(7) DEFAULT 0', 'appear_annot': 'VARCHAR(7) DEFAULT 0', 'disappear_annot': 'VARCHAR(7) DEFAULT 0', 'division_annot': 'VARCHAR(7) DEFAULT 0'}
Added missing column: frontier
Added missing column: height
Added missing column: features
Added missing column: node_prob


# adding node

In [ ]:
node_id = 175000014
time = 176

pickle = get_node_values(config_adjusted.data_config, [int(node_id)], NodeDB.pickle)
print(pickle)

new_id = add_new_node(config_adjusted,
                        time = time,
                        mask = pickle.mask,
                        bbox = pickle.bbox,
                        include_overlaps=False,
                        )

index: 114946
bbox: [ 20 645 200  56 714 372]
area: 51215.0
centroid: [ 39 679 296]



In [5]:
from ultrack.core.segmentation.node import Node

node = Node.from_mask(
        time=time,
        mask=pickle.mask,
        bbox=np.asarray(pickle.bbox),
    )

print(node.area)
print(type(node.area))

51215
<class 'numpy.int64'>
